In [183]:
import pandas as pd
import psycopg2
from config_postgresql import database, user, password, port, host

In [184]:
connection = psycopg2.connect(database=database, user = user, password=password, port=port, host=host)
connection.autocommit = True

In [617]:
def sql(request):
    cursor = connection.cursor()
    cursor.execute(request)
    result = cursor.fetchall()
    columns = []
    for desc in cursor.description:
        columns.append(desc[0])
    
    return pd.DataFrame(result, columns=columns)

In [6]:
sql('''SELECT 
    CONCAT(module_id, ' ', LEFT(CONCAT(module_name), 14), '...') AS Модуль,
    CONCAT(module_id, '.', lesson_position, ' ', LEFT(CONCAT(lesson_name), 12), '...') AS Урок,
    CONCAT(module_id, '.', lesson_position, '.', step_position, step_name) AS Шаг
FROM
    module
    INNER JOIN lesson USING(module_id)
    INNER JOIN step USING(lesson_id)
WHERE step_name LIKE '%вложен%запрос%'
ORDER BY Модуль, Урок, Шаг;''')

,Модуль,Урок,Шаг
0,1 Основы реляцио...,1.4 Вложенные за...,1.4.3Использование вложенного запроса в выражении
1,1 Основы реляцио...,1.4 Вложенные за...,1.4.3Использование вложенного запроса в выражении
2,1 Основы реляцио...,1.5 Запросы корр...,"1.5.5Добавление записей, вложенные запросы"
3,1 Основы реляцио...,1.5 Запросы корр...,"1.5.5Добавление записей, вложенные запросы"
4,2 Запросы SQL к ...,2.2 Запросы на в...,2.2.7Запросы для нескольких таблиц со вложенны...
5,2 Запросы SQL к ...,2.2 Запросы на в...,2.2.7Запросы для нескольких таблиц со вложенны...
6,2 Запросы SQL к ...,2.3 Запросы корр...,"2.3.5Запрос на обновление, вложенные запросы"
7,2 Запросы SQL к ...,2.3 Запросы корр...,"2.3.5Запрос на обновление, вложенные запросы"


### 1. Вывести первых 10 студентов с самым большим числом правильных ответов.

In [89]:
sql('''select student_id, count(result) as Количество
from step_student
where result = 'correct'
group by student_id
order by Количество desc
limit 10''')

,student_id,Количество
0,53,68
1,60,66
2,36,62
3,9,62
4,46,62
5,30,62
6,39,62
7,15,62
8,59,60
9,4,60


### 2. Выведите топ-20 студентов по объему выполненных заданий, добавить столбцы -  % пройденного материала, среднее время решения задания, количество неправильных попыток.

**Решение:**  
    1. Выводим таблицу с условием 'correct'.  
    2. Выводим таблицу с условием 'wrong'.  
    3. Формируем запрос, считаем процент.  
    4. Джойним таблицу для вывода имени студентов, делаем сортировку по проценту и ставим лимит.  

In [287]:
sql('''
with correct_raiting (student_id, count_step, avg_sec) as
    (
        select student_id, count(distinct step_id) as count_step,
        round(avg(submission_time - attempt_time)) as avg_sec
        from step_student
        where result = 'correct'
        group by student_id
    ),
wrong_raiting (student_id, count_wrong) as
    (
        select student_id, count(result) as count_wrong
        from step_student
        where result = 'wrong'
        group by student_id    
    )
select student_name, 
(count_step*100/(select count(distinct step_id) from step_student)) as percent, count_wrong, avg_sec 
from correct_raiting
    join wrong_raiting using(student_id)
    join student using(student_id)
order by percent desc
limit 20
''')

,student_name,percent,count_wrong,avg_sec
0,student_60,96,9,1984
1,student_43,93,140,22954
2,student_18,93,11,36
3,student_27,93,50,6935
4,student_49,93,9,8508
5,student_46,93,2,5771
6,student_31,93,3,63876
7,student_39,93,9,747
8,student_30,93,1,19014
9,student_36,93,105,4299


### 3. После каких шагов курса прекратило обучение больше всего студентов, сориентируйте, исходя из соотвествующих модулей, -  это начало, середина или  конец курса.

Решение:
1. Делаем группировку по студентам и последнему шагу у каждого.
2. Выбираем из этой таблицы количесто шагов = количество студентов, делаем группировку.
3. Джойним таблицы, чтобы дойти до модуля.
4. Прописываем условия для модуля.
5. Группируем по шагам, модулю и задаем сортировку.

In [301]:
sql('''
select step_name,
    case
        when module_id = 1 then 'Начало'
        when module_id = 2 then 'Середина'
        when module_id = 3 then 'Конец'
    end as Прогресс, count(last_step) as count_student
from
    (
        select student_id, max(step_id) as last_step from step_student
        group by student_id
    ) as t1
join step on step.step_id = t1.last_step
join lesson using(lesson_id)
group by step_name, module_id
order by count_student desc
        ''')

,step_name,Прогресс,count_student
0,Выборка данных с сортировкой,Начало,25
1,Задание. Вывести самый популярный жанр,Середина,18
2,"Выборка данных, оператор LIKE",Начало,3
3,Запросы для нескольких таблиц с группировкой,Середина,3
4,"Выборка данных, операторы BETWEEN, IN",Начало,3
5,"Выборка данных, логические операции",Начало,2
6,"Операция соединение, использование USING()",Середина,2
7,Внешнее соединение LEFT и RIGHT OUTER JOIN,Середина,2
8,"Задание. Работа с архивной таблицей, оператор ...",Середина,1
9,Перекрестное соединение CROSS JOIN,Середина,1


### 4. Какие студенты занимались в праздники - за праздники взять только те даты, которые подпадают под официальные выходные

Решение:

1. Делаем рекурсивную таблицу с днями занятий, объединяем в один столбец дату начала и окончания выполнения задания.
2. Меняем нужные даты на праздники.
3. Делаем основной запрос, откуда убираем все нулевые значения, сортируем.

In [378]:
sql(''' 
with date_table(student_id, date_1) as
    (
        select student_id, (to_char(to_timestamp(attempt_time), 'mm-dd')) as date_1 from step_student
        union
        select student_id, (to_char(to_timestamp(submission_time), 'mm-dd')) as date_1 from step_student
    )
select date_1 as holydays, student_id from 
    (
        select 
            case 
                when date_1 = '01-01' then 'Новый год'
                when date_1 = '01-07' then 'Рождество'
                when date_1 = '02-23' then '23 февраля'
                when date_1 = '03-08' then '8 марта'
                when date_1 = '05-01' then '1 мая'
                when date_1 = '09-01' then 'День Победы'
                when date_1 = '07-12' then 'День России'
                when date_1 = '04-11' then 'День народного единства'
            end as date_1, student_id from date_table
    ) as holydays_table
where date_1 is not null
order by holydays
''')

,holydays,student_id
0,1 мая,9
1,1 мая,28
2,День Победы,60
3,День Победы,52
4,День Победы,61
5,День России,8
6,День России,13
7,День России,55


### 5. Составьте рейтинг пользователей, кто прошел быстрее всех задания с sql-запросами по каждому из шагов по базе данных 'Абитуриент'

In [591]:
sql('''
with abiturient(step_name, step_id, lesson_id) as
    (
        select step_name, step_id, lesson_id from step
        join lesson using(lesson_id)
        where (lesson_id = 11 or lesson_id = 12)
        and (step_type = 'sql')
        and (step_name not in ('Запросы пользователей'))
    ) 

select step_name, student_name, time_sec, 
dense_rank() OVER (PARTITION BY step_name ORDER BY time_sec) as rating
from 
    (
        select distinct on (step_name, student_id, result, step_id) step_name, student_id, result, step_id,
        (submission_time - attempt_time) as time_sec from 
            (
                select * from step_student
                order by attempt_time
            ) as t1
        join abiturient using(step_id)
        where result = 'correct'
        group by step_name, student_id, result, step_id, time_sec
    ) as t2
join student using(student_id)
order by step_id, time_sec
limit 30
''')

,step_name,student_name,time_sec,rating
0,Запросы на основе трех и более связанных таблиц,student_36,5,1
1,Запросы на основе трех и более связанных таблиц,student_23,11,2
2,Запросы на основе трех и более связанных таблиц,student_51,11,2
3,Запросы на основе трех и более связанных таблиц,student_18,13,3
4,Запросы на основе трех и более связанных таблиц,student_46,16,4
5,Запросы на основе трех и более связанных таблиц,student_39,16,4
6,Запросы на основе трех и более связанных таблиц,student_24,18,5
7,Запросы на основе трех и более связанных таблиц,student_43,24,6
8,Запросы на основе трех и более связанных таблиц,student_9,96,7
9,Запросы на основе трех и более связанных таблиц,student_53,101,8


### 6. Составьте рейтинг усердных пользователей, кто дал сделал больше всего неправильных попыток, но в итоге выполнил задание.

In [572]:
sql('''
select * from step_student
limit 3
''')

,step_student_id,step_id,student_id,attempt_time,submission_time,result
0,1,10,52,1598291444,1598291490,correct
1,2,10,11,1593291995,1593292031,correct
2,3,10,19,1591017571,1591017743,correct


In [634]:
sql('''
select step_name, student_name, attempts from 
    (
        select distinct on (student_id, step_id) student_id, step_id, result, attempts
        from 
            (
                select student_id, step_id, result, attempt_time, 
                dense_rank() OVER (PARTITION BY student_id, step_id ORDER BY attempt_time) as attempts
                from step_student
                order by student_id, step_id
            ) as t1
        where attempts > 1 and result='correct'
        order by student_id, step_id, attempts desc
    ) as t2
join step using(step_id)
join student using(student_id)
order by attempts desc
''')

,step_name,student_name,attempts
0,"Выборка данных, оператор LIKE",student_27,41
1,Запросы для нескольких таблиц со вложенными за...,student_43,32
2,Запросы для нескольких таблиц с группировкой,student_36,27
3,"Выборка данных, оператор LIKE",student_23,22
4,Выборка данных с сортировкой,student_40,19
...,...,...,...
354,"Выборка данных, логические операции",student_22,2
355,"Выборка данных, логические операции",student_28,2
356,"Выборка данных, логические операции",student_32,2
357,"Выборка данных, оператор LIKE",student_33,2


In [633]:
sql('''select * from step_student
where student_id = 27 and step_id = 19
order by attempt_time''')

,step_student_id,step_id,student_id,attempt_time,submission_time,result
0,1124,19,27,1596633406,1596640413,wrong
1,899,19,27,1596640423,1596640431,wrong
2,891,19,27,1596640437,1596640444,wrong
3,884,19,27,1596640449,1596640453,wrong
4,883,19,27,1596640493,1596640498,wrong
5,881,19,27,1596640529,1596640535,wrong
6,915,19,27,1596640590,1596640688,wrong
7,872,19,27,1596640699,1596640703,wrong
8,921,19,27,1596640763,1596640769,wrong
9,904,19,27,1596640808,1596640816,wrong
